In [1]:
## 데이터베이스를 사용하기 위해 Python DB API 표준을 따르는 pymysql이라는 모듈 사용
import pymysql
import csv

import pandas as pd
import numpy as np
import re

# 형태소 분석기
from konlpy.tag import *
hannanum = Hannanum()

In [2]:
# pandas 라이브러리를 이용해서 엑셀을 데이터프레임형으로 불러옴
df_frame = pd.read_excel('mpva_event1012.xlsx')
# column명 변경
df_frame.columns = ['MNG_NO', 'EVENT_NO', 'DATE_NO', '사건']
df_frame

,MNG_NO,EVENT_NO,DATE_NO,사건
0,950430,1,19220200,박열과 의기투합하여 일본 제국주의에 반대하는 활동을 시작
1,950430,2,19220500,흑도회에 가입
2,950430,3,19220900,박열과 함께 기관지 흑도의 발간책임을 맡음
3,950430,4,19220900,흑우회를 조직해 사상연구와 연설회 등을 추진하면서 기관지 민중 운동을 한글로 발행
4,950430,5,19230400,무정부주의 운동가들을 규합하여 불령사를 조직
5,950430,6,19230900,보호감속 명목으로 도쿄 세타가야 경찰서에 구금
6,950430,7,19260325,대심원 제1특별혀사부에서 이른바 형법 제73조 및 폭발물취제규칙위반 등으로 사형을 ...
7,950430,8,19260400,무기징역으로 감형
8,950430,9,19260723,옥중에서 순국
9,888888,1,19990000,경기도 고양시


In [3]:
### 1차, 2차로 나누어 코드가 진행된다. 1차에서 대부분의 틀이 만들어지며 오류가 있어 2차에서 수정한다.###
### 1차 : 만들고자 하는 Dataframe형의 표를 만드는 과정 ###

# '사건' column을 선택함
df_evelist = df_frame[['사건']].values.tolist()
# 리스트형에서 문자형으로 변환하는 코드
ListoStr = ''.join([str(element) for element in df_evelist[0] ])
mpva_evenouns = hannanum.nouns(ListoStr)

# 첫번째 row 출력
df_1row = df_frame[0:1]
# 형태소 분석기를 통해 나온 단어들로새로운 dataframe형을 만듬
df_evecols = pd.DataFrame(mpva_evenouns)
# column명 'Word'로 선언
df_evecols.columns = ['WORD']

# 기존 행과 새로 만든 컬럼의 합성 ( df_1row와 df_evecols의 결합 )
syn_df = pd.concat([df_1row, df_evecols], axis= 1).fillna(method='pad').drop('사건', axis =1)
syn_df = syn_df.astype({'MNG_NO': 'int'}).astype({'EVENT_NO': 'int'}).astype({'DATE_NO': 'int'})

# 지속적으로 다음 행들을 추가하기 위해 만든 반복문
# append 함수를 통해 다음 행의 값들 추가
# df_frame column 'MNG_NO'의 갯수만큼 범위를 지정함. (= 엑셀 행의 갯수와 같다)
for i in range(1, df_frame['MNG_NO'].count()):
    ListoStr2 = ''.join([str(element) for element in df_evelist[i] ])
    mpva_evenouns = hannanum.nouns(ListoStr2)

    # index i번째 줄의 row
    df_2row = df_frame[i:i+1]
    df_2row.index = ['0']

    df_evecols2 = pd.DataFrame(mpva_evenouns)
    df_evecols2.columns = ['WORD']

    syn_df2 = pd.concat([df_2row, df_evecols2], axis= 1).fillna(method='pad').drop('사건', axis =1)
    syn_df2 = syn_df2.astype({'MNG_NO': 'int'}).astype({'EVENT_NO': 'int'}).astype({'DATE_NO': 'int'})

    # 결측치 제거
    syn_df2 = syn_df2.dropna()

    # 행 값 추가
    # ignore_index=True : Index 이름을 무시
    syn_df = syn_df.append(syn_df2, ignore_index=True)
    
    # 연번 생성
    syn_df["SERIAL_NO"] = syn_df.index + 1

    # column 순서, 위치 설정
    syn_df = syn_df[['MNG_NO', 'EVENT_NO', 'SERIAL_NO', 'DATE_NO', 'WORD']]
    
syn_df   

,MNG_NO,EVENT_NO,SERIAL_NO,DATE_NO,WORD
0,950430,1,1,19220200,박열
1,950430,1,2,19220200,의기투합
2,950430,1,3,19220200,일본
3,950430,1,4,19220200,제국주의
4,950430,1,5,19220200,반대
5,950430,1,6,19220200,활동
6,950430,1,7,19220200,시작
7,950430,2,8,19220500,흑도회
8,950430,2,9,19220500,가입
9,950430,3,10,19220900,박열


In [4]:
# 2차 : SERIAL_NO의 값이 알맞게 들어가지 않기 때문에 수정해준다.
# 수정하기 위한 반복문
# 이미 만들어진 syn_df를 이용하여 최종적인 for문을 완성시킨다.


# MNG_NO 값별로의 데이터 갯수를 count() 함수를 이용하여 세기
# 즉, 중복되지 않은 MNG_NO 값별
for j in range(0, df_frame['MNG_NO'].value_counts().count()): 
    
    # copy() : 원천 Dataframe과 별도인 메모리 공간을 파생 Dataframe에 부여하는 것, 
    #          파생 Dataframe에 메모리가 부여되므로 사용 가능한 총 메모리는 줄어들게 된다.
    # 요약 : 메모리 부담 줄임
    # DataFrame.drop_duplicates() : 중복값 처리(unique한 1개의 key만 남기고 나머지 중복은 제거)

    # 밑의 코드를 쉽게 표현하면 df_example = df[0:1].copy()

    # 이미 syn_df에서 만든 dataframe의 범위를 지정해서 final_df_syn으로 다시 재정의한 것
    # 범위 : j번째 MNG_NO의 첫 행 값부터 MNG_NO 번호 1개마다 생성된 'Word'column의 row 갯수까지
    final_df_syn = syn_df[syn_df['MNG_NO'].drop_duplicates().index[j]:syn_df['MNG_NO'].drop_duplicates().index[j]+
                      syn_df['MNG_NO'].drop_duplicates(keep='last').index[j]-syn_df['MNG_NO'].drop_duplicates().index[j]+1].copy()
    
    # 한번 더 반복문을 돌려서 Serial_NO의 값들을 위치에 맞게 재설정함
    k=0
    for k in range(0, syn_df['MNG_NO'].drop_duplicates(keep='last').index[j]-syn_df['MNG_NO'].drop_duplicates().index[j]+1):
        # loc 함수로 조건을 설정하여 연번을 다시 재설정함
        final_df_syn.loc[final_df_syn['SERIAL_NO'] == syn_df['MNG_NO'].drop_duplicates().index[j] + (k+1), 'SERIAL_NO'] = k + 1
        # print(syn_df['MNG_NO'].drop_duplicates().index[j] + (k+1))
    
    print(final_df_syn)

    MNG_NO  EVENT_NO  SERIAL_NO   DATE_NO       WORD
0   950430         1          1  19220200         박열
1   950430         1          2  19220200       의기투합
2   950430         1          3  19220200         일본
3   950430         1          4  19220200       제국주의
4   950430         1          5  19220200         반대
5   950430         1          6  19220200         활동
6   950430         1          7  19220200         시작
7   950430         2          8  19220500        흑도회
8   950430         2          9  19220500         가입
9   950430         3         10  19220900         박열
10  950430         3         11  19220900        기관지
11  950430         3         12  19220900         흑도
12  950430         3         13  19220900        발간책
13  950430         4         14  19220900        흑우회
14  950430         4         15  19220900         조직
15  950430         4         16  19220900       사상연구
16  950430         4         17  19220900        연설회
17  950430         4         18  19220900     

In [5]:
###########################################################################################################################
# Unnecessary Code
###########################################################################################################################

In [6]:
# 이해하는데 도움될 만한(?) 삽질코드들

In [7]:
# 불러오는 엑셀의 column 중 관리번호만 사용함
dfcol_count_123 = pd.read_excel('mpva_event1012.xlsx', usecols=['관리번호'])
dfcol_count_123

,관리번호
0,950430
1,950430
2,950430
3,950430
4,950430
5,950430
6,950430
7,950430
8,950430
9,888888


In [8]:
# 엑셀 행의 갯수
df_frame['MNG_NO'].count()

13

In [9]:
# MNG_NO 값별로의 데이터 갯수
df_frame['MNG_NO'].value_counts()

950430    9
888888    3
777777    1
Name: MNG_NO, dtype: int64

In [10]:
df_frame['MNG_NO'].value_counts().count()

3

In [11]:
# reset_index() : 인덱스 숫자를 재배열할 때 사용함
df_frame['MNG_NO'].value_counts().reset_index().drop('index', axis=1)

,MNG_NO
0,9
1,3
2,1


In [12]:
# 삽질
df_indexTest = df_frame['MNG_NO'].value_counts(ascending=True).reset_index().drop('index', axis=1).index[0]
df_indexTest

0

In [13]:
for z in range(0, df_frame['MNG_NO'].value_counts().count() ):
    df_frame['MNG_NO'].value_counts().index[z]
    print(df_frame['MNG_NO'].value_counts().index[z])

950430
888888
777777


In [14]:
# DataFrame.duplicated() :중복 여부 확인
# DataFrame.drop_duplicates() : 중복값 처리(unique한 1개의 key만 남기고 나머지 중복은 제거)
df_1111 = syn_df.loc[syn_df['EVENT_NO'] == 1].drop_duplicates()
df_2222 = df_1111['MNG_NO'].drop_duplicates()

print( syn_df.loc[syn_df['EVENT_NO'] == 1].drop_duplicates() ) 
print( df_1111['MNG_NO'].drop_duplicates() )

    MNG_NO  EVENT_NO  SERIAL_NO   DATE_NO  WORD
0   950430         1          1  19220200    박열
1   950430         1          2  19220200  의기투합
2   950430         1          3  19220200    일본
3   950430         1          4  19220200  제국주의
4   950430         1          5  19220200    반대
5   950430         1          6  19220200    활동
6   950430         1          7  19220200    시작
46  888888         1         47  19990000    경기
47  888888         1         48  19990000   고양시
52  777777         1         53  19992222   흑도회
53  777777         1         54  19992222    가입
0     950430
46    888888
52    777777
Name: MNG_NO, dtype: int32


In [15]:
# 처음 MNG_NO = 0 값일 때의 첫 행 값
print(syn_df['MNG_NO'].drop_duplicates().index[j-1])
# 처음 MNG_NO = 0 값일 때의 마지막 행 값
print(syn_df['MNG_NO'].drop_duplicates(keep='last').index[j-1])
# index[0]: 0, index[1]: 46, index[2]: 52
print(syn_df['MNG_NO'].drop_duplicates().index[0])
print('-')
# 2번째 MNG_NO의 첫 행 값
print(syn_df['MNG_NO'].drop_duplicates().index[j])
# 2번째 MNG_NO의 마지막 행 값
print(syn_df['MNG_NO'].drop_duplicates(keep='last').index[j])
print('-')
df_1111 = syn_df.loc[syn_df['EVENT_NO'] == 1].drop_duplicates()
print(syn_df.loc[syn_df['EVENT_NO'] == 1].drop_duplicates()) 
print()
print(df_1111['MNG_NO'].drop_duplicates())

46
51
0
-
52
53
-
    MNG_NO  EVENT_NO  SERIAL_NO   DATE_NO  WORD
0   950430         1          1  19220200    박열
1   950430         1          2  19220200  의기투합
2   950430         1          3  19220200    일본
3   950430         1          4  19220200  제국주의
4   950430         1          5  19220200    반대
5   950430         1          6  19220200    활동
6   950430         1          7  19220200    시작
46  888888         1         47  19990000    경기
47  888888         1         48  19990000   고양시
52  777777         1         53  19992222   흑도회
53  777777         1         54  19992222    가입

0     950430
46    888888
52    777777
Name: MNG_NO, dtype: int32


In [16]:
df_333row = syn_df[syn_df['MNG_NO'].drop_duplicates().index[j]:syn_df['MNG_NO'].drop_duplicates().index[j]+1]
print(df_333row)
# 엑셀에 표기되어있는 중복된 관리번호 행의 갯수 구하는 식
print(syn_df['MNG_NO'].drop_duplicates(keep='last').index[j]-syn_df['MNG_NO'].drop_duplicates().index[j]+1)

    MNG_NO  EVENT_NO  SERIAL_NO   DATE_NO WORD
52  777777         1         53  19992222  흑도회
2


In [17]:
# ★★★ 여기부터는 별로 도움 안될 수 있는 코드들이라 지워도 무관 ★★★
# 알맞은 인덱스 값들의 범위를 구하기 위해 만든 코드
# 주요 코드들은 아니라 별도의 주석은 첨부 안함

In [18]:
# row 총 갯수를 구하는 코드
syn_row = syn_df.shape[0]
last_syn_row = syn_row - 1
syn_df_index_int = syn_df['MNG_NO'].drop_duplicates(keep='last').index[j-1].astype('int64')
if( syn_df_index_int == last_syn_row ):
    print(syn_df['MNG_NO'].drop_duplicates(keep='last').index[j-1])

In [19]:
# Dataframe for문을 돌리기 위해 만든 예시코드
for z in range(0, df_frame['MNG_NO'].value_counts().count() ):
    
    # 각각 MNG_NO의 첫 행을 의미하는 인덱스값 z
    df_frame['MNG_NO'].drop_duplicates().index[z]
    # 각각 MNG_NO의 마지막 행을 의미하는 인덱스값 z
    print(df_frame['MNG_NO'].drop_duplicates().index[z])
    
    df_frame['MNG_NO'].drop_duplicates(keep='last').index[z]
    print(df_frame['MNG_NO'].drop_duplicates(keep='last').index[z])
    
#     print(df_frame['MNG_NO'].drop_duplicates())
    
#     df_frame['MNG_NO'].drop_duplicates().index[z]:df_frame['MNG_NO'].drop_duplicates(keep='last').index[z]
    
#     range(df_frame['MNG_NO'].drop_duplicates().index[i], df_frame['MNG_NO'].drop_duplicates(keep='last').index[i])
    
#     df_frame[['MNG_NO','EVENT_NO']].drop_duplicates(keep='last').index[z]
#     print(df_frame[['MNG_NO','EVENT_NO']].drop_duplicates(keep='last').index[z])
    
#     1~9
#     df_frame[df_frame['MNG_NO'].drop_duplicates().index[z]:df_frame['MNG_NO'].drop_duplicates(keep='last').index[z]+1]
#     print(df_frame[df_frame['MNG_NO'].drop_duplicates().index[z]:df_frame['MNG_NO'].drop_duplicates(keep='last').index[z]+1])

    df_frame[df_frame['MNG_NO'].drop_duplicates().index[z]:df_frame['MNG_NO'].drop_duplicates().index[z]+1]
    print(df_frame[df_frame['MNG_NO'].drop_duplicates().index[z]:df_frame['MNG_NO'].drop_duplicates().index[z]+1])
    
    print('다음')
    print('')

0
8
   MNG_NO  EVENT_NO   DATE_NO                               사건
0  950430         1  19220200  박열과 의기투합하여 일본 제국주의에 반대하는 활동을 시작
다음

9
11
   MNG_NO  EVENT_NO   DATE_NO       사건
9  888888         1  19990000  경기도 고양시
다음

12
12
    MNG_NO  EVENT_NO   DATE_NO       사건
12  777777         1  19992222  흑도회에 가입
다음



In [20]:
for z in range(0, df_frame['MNG_NO'].value_counts().count() ):
    df_evelist = df_frame[['사건']].values.tolist()
    ListoStr = ''.join([str(element) for element in df_evelist[df_frame['MNG_NO'].drop_duplicates().index[z]] ])
    mpva_evenouns = hannanum.nouns(ListoStr)
    print('첫번째')
    print(mpva_evenouns)
    
    df_1row = df_frame[df_frame['MNG_NO'].drop_duplicates().index[z]:df_frame['MNG_NO'].drop_duplicates().index[z]+1]
    print('두번째')
    print(df_1row)  
    
    df_evecols = pd.DataFrame(mpva_evenouns)
    df_evecols.columns = ['WORD']
    print('세번째')
    print(df_evecols)
    
    print('네번째')
    print(df_frame['MNG_NO'].drop_duplicates().index[z])
    print('--------------------------')

첫번째
['박열', '의기투합', '일본', '제국주의', '반대', '활동', '시작']
두번째
   MNG_NO  EVENT_NO   DATE_NO                               사건
0  950430         1  19220200  박열과 의기투합하여 일본 제국주의에 반대하는 활동을 시작
세번째
   WORD
0    박열
1  의기투합
2    일본
3  제국주의
4    반대
5    활동
6    시작
네번째
0
--------------------------
첫번째
['경기', '고양시']
두번째
   MNG_NO  EVENT_NO   DATE_NO       사건
9  888888         1  19990000  경기도 고양시
세번째
  WORD
0   경기
1  고양시
네번째
9
--------------------------
첫번째
['흑도회', '가입']
두번째
    MNG_NO  EVENT_NO   DATE_NO       사건
12  777777         1  19992222  흑도회에 가입
세번째
  WORD
0  흑도회
1   가입
네번째
12
--------------------------


In [21]:
# 삽질 for문, SERIAL_NO 값을 어떻게 수정해야할 지 임의적으로 선언한 코드(garbage code)
# syn_df.loc[ 46 ,'SERIAL_NO'] = idx + 1
# syn_df.loc[ 47 ,'SERIAL_NO'] = idx + 2

df_evelist = df_frame[['사건']].values.tolist()
ListoStr = ''.join([str(element) for element in df_evelist[0] ])
mpva_evenouns = hannanum.nouns(ListoStr)

df_1row = df_frame[0:1]
df_evecols = pd.DataFrame(mpva_evenouns)
df_evecols.columns = ['WORD']

syn_df = pd.concat([df_1row, df_evecols], axis= 1).fillna(method='pad').drop('사건', axis =1)
syn_df = syn_df.astype({'MNG_NO': 'int'}).astype({'EVENT_NO': 'int'}).astype({'DATE_NO': 'int'})

for i in range(1, df_frame['MNG_NO'].count()):
    ListoStr2 = ''.join([str(element) for element in df_evelist[i] ])
    mpva_evenouns = hannanum.nouns(ListoStr2)

    df_2row = df_frame[i:i+1]
    df_2row.index = ['0']

    df_evecols2 = pd.DataFrame(mpva_evenouns)
    df_evecols2.columns = ['WORD']

    # axis= 0 : 행방향, axis = 열 방향
    syn_df2 = pd.concat([df_2row, df_evecols2], axis= 1).fillna(method='pad').drop('사건', axis =1)
    syn_df2 = syn_df2.astype({'MNG_NO': 'int'}).astype({'EVENT_NO': 'int'}).astype({'DATE_NO': 'int'})

    # 결측치 제거
    syn_df2 = syn_df2.dropna()

    if(i < 9) :
        syn_df = syn_df.append(syn_df2, ignore_index=True)
        syn_df["SERIAL_NO"] = syn_df.index + 1
        syn_df = syn_df[['MNG_NO', 'EVENT_NO', 'SERIAL_NO', 'DATE_NO', 'WORD']]
    if(i >= 9 and i <= 11) :
        syn_df = syn_df.append(syn_df2, ignore_index=True)
        idx = 0
        syn_df.loc[ 46 ,'SERIAL_NO'] = idx + 1
        syn_df.loc[ 47 ,'SERIAL_NO'] = idx + 2

syn_df

,MNG_NO,EVENT_NO,SERIAL_NO,DATE_NO,WORD
0,950430,1,1.0,19220200,박열
1,950430,1,2.0,19220200,의기투합
2,950430,1,3.0,19220200,일본
3,950430,1,4.0,19220200,제국주의
4,950430,1,5.0,19220200,반대
5,950430,1,6.0,19220200,활동
6,950430,1,7.0,19220200,시작
7,950430,2,8.0,19220500,흑도회
8,950430,2,9.0,19220500,가입
9,950430,3,10.0,19220900,박열
